# OpenAI 외 도구 호출 에이전트(Tool Calling Agent)

OpenAI 외에도 `Anthropic`, `Google Gemini`, `Together.ai`, `Ollama`, `Mistral`과 같은 더 광범위한 공급자 구현을 지원합니다.

이번 챕터에서는 다양한 LLM 을 사용하여 도구 호출 에이전트를 생성하고 실행하는 방법을 살펴보겠습니다.

**참고 링크**

- [LangChain 공식 도큐먼트](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/)

In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [3]:
from datetime import datetime
import requests
from langchain.tools import tool


# 도구 정의
@tool
def get_current_time():
    """Use this to get current date or time"""
    return datetime.now()

@tool
def search_dayoff(start_date: str, end_date: str) -> str:
    """
    특정 날짜의 휴가 일정을 조회합니다.

    Args:
        start_date (str): 시작일, YYYY-MM-DD 형식의 날짜 (예: 2024-01-01)
        end_date (str): 종료일, YYYY-MM-DD 형식의 날짜 (예: 2024-01-01)

    Returns:
        str: 해당 날짜의 휴가 일정 정보
    """
    try:
        url = "http://172.16.120.203:9201/flex_dayoff_calendar/_search"
        body = {"query": {"range": {"date": {"gte": start_date, "lte": end_date}}}}
        response = requests.post(url, json=body)

        if response.status_code == 200:
            data = response.json()
            hits = data.get("hits", {}).get("hits", [])
            
            if hits:
                result = []
                for hit in hits:
                    source = hit.get("_source", {})
                    result.append(
                        f"[날짜] {source.get('date', '정보 없음')}\n "
                        f"[휴가자] {source.get('text', '정보 없음')}, "
                    )
                return "\n".join(result)
            else:
                return f"해당하는 휴가 일정이 없습니다."
        else:
            return f"API 호출 실패: HTTP {response.status_code}"

    except ValueError:
        return "잘못된 날짜 형식입니다. YYYY-MM-DD 형식으로 입력해주세요."
    except Exception as e:
        return f"오류가 발생했습니다: {str(e)}"

In [4]:
search_dayoff.invoke({"start_date": "2025-03-31", "end_date": "2025-03-31"})

"[날짜] 2025-03-31\n [휴가자] ['🌴 [정지훈] 휴가', '🌴 [정은영] 휴가', '🌴 [김지훈] 휴가', '🌴 [김건우] 휴가 - 3:30\\u202fPM ~ 5:30\\u202fPM', '🌴 [제은비] 휴가'], "

In [5]:
# tools 정의
tools = [get_current_time, search_dayoff]

## Agent 용 프롬프트 생성

- `chat_history` : 이전 대화 내용을 저장하는 변수 (멀티턴을 지원하지 않는다면, 생략 가능합니다.)
- `agent_scratchpad` : 에이전트가 임시로 저장하는 변수
- `input` : 사용자의 입력

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "use the `get_current_time` tool to find out today's date."
            "Make sure to use the `search_dayoff` tool for searching 연차, 휴가."
            "",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

## Tool Calling 을 지원하는 다양한 LLM 목록

실습 진행을 위해서는 아래 내용을 설정해야 합니다.

**Anthropic**

- [Anthropic API 키 발급 관련](https://console.anthropic.com/settings/keys)
- `.env` 파일 내 `ANTHROPIC_API_KEY` 에 발급받은 키를 설정하세요

**Gemini**

- [Gemini API 키 발급 관련](https://aistudio.google.com/app/apikey?hl=ko)
- `.env` 파일 내 `GOOGLE_API_KEY` 에 발급받은 키를 설정하세요

**Together AI**

- [Together AI API 키 발급 관련](https://api.together.ai/)
- `.env` 파일 내 `TOGETHER_API_KEY` 에 발급받은 키를 설정하세요

**Ollama**

- [Ollama Tool Calling 지원 모델 리스트](https://ollama.com/search?c=tools)
- [이번 실습에 사용할 llama3.1 모델](https://ollama.com/library/llama3.1)
- 터미널 창에 `ollama pull llama3.1` 명령어를 입력하여 모델을 다운로드 받습니다.
- 이전에 Ollama 를 사용하지 않았다면, [Ollama](https://wikidocs.net/233805) 를 참고해 주세요.

langchain-ollama 설치를 한 뒤 진행해 주세요.

In [ ]:
# !pip install -qU langchain-ollama==0.1.3

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
import os

# GPT-4o-mini
gpt = ChatOpenAI(model="gpt-4o-mini")

# Claude-3-5-sonnet
claude = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)

# Gemini-1.5-pro-latest
gemini = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

# Llama-3.1-70B-Instruct-Turbo
llama = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
)

gemma3 = ChatOllama(model="gemma-3:latest", temperature=0)

qwen = ChatOllama(model="qwen-2.5:latest") 


In [ ]:
llama_gpt = ChatOpenAI(
    model="meta-llama/Llama-3.1-8B-Instruct",
    base_url="https://gqei2ikplhhuvm-8000.proxy.runpod.net/v1")

In [ ]:
llm_with_llama = llama_gpt.bind_tools(tools)

In [ ]:
llm_with_llama.invoke("오늘은 몇일인가요?")

LLM 기반으로 Agent 를 생성합니다.

In [ ]:
from langchain.agents import create_tool_calling_agent

# Agent 생성
gpt_agent = create_tool_calling_agent(gpt, tools, prompt)
claude_agent = create_tool_calling_agent(claude, tools, prompt)
gemini_agent = create_tool_calling_agent(gemini, tools, prompt)
llama_agent = create_tool_calling_agent(llama, tools, prompt)
gemma_agent = create_tool_calling_agent(gemma3, tools, prompt)
qwen_agent = create_tool_calling_agent(qwen, tools, prompt)

## AgentExecutor 생성 후 실행 및 결과 확인



In [ ]:
from langchain.agents import AgentExecutor

# gpt_agent 실행
agent_executor = AgentExecutor(
    agent=gpt_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

# result = agent_executor.invoke({"input": "오늘 휴가자 누구야?"})
result = agent_executor.invoke({"input": "What day is it today?"})
# result = agent_executor.invoke({"input": "이번주 휴가자 모두 말해줘"})

print("Agent 실행 결과:")
print(result["output"])

다양한 llm을 사용하여 에이전트를 실행합니다.

다음은 입력받은 llm을 사용하여 Agent 를 생성하고 실행하여 결과를 출력하는 함수입니다.

In [ ]:
def execute_agent(llm, tools, input_text, label):
    agent = create_tool_calling_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
    result = executor.invoke({"input": input_text})
    print(f"[{label}] 결과입니다.")
    if isinstance(result["output"], list) and len(result["output"]) > 0:
        for item in result["output"]:
            if "text" in item:
                print(item["text"])
    elif isinstance(result["output"], str):
        print(result["output"])
    else:
        print(result["output"])

각 llm 별로 에이전트를 생성하고 실행하여 결과를 출력합니다.

In [ ]:
query = (
    "오늘 휴가자 누구야?"
)

In [35]:
# gpt
execute_agent(gpt, tools, query, "gpt")

[gpt] 결과입니다.
오늘 휴가자는 다음과 같습니다:

- 🌴 정지훈 휴가
- 🌴 문승만 휴가 (2:00 PM ~ 5:00 PM)
- 🌴 김건우 휴가 (1:00 PM ~ 5:00 PM)
- 🌴 이선영 휴가
- 🌴 천승민 휴가 (9:00 AM ~ 2:00 PM)
- 🌴 김지훈 휴가 (9:00 AM ~ 2:00 PM)


In [30]:
# claude
execute_agent(claude, tools, query, "claude")

[claude] 결과입니다.
오늘(2025년 4월 7일)의 휴가자 정보는 다음과 같습니다:

1. 정지훈: 종일 휴가
2. 문승만: 오후 2시부터 5시까지 휴가
3. 김건우: 오후 1시부터 5시까지 휴가
4. 이선영: 종일 휴가
5. 천승민: 오전 9시부터 오후 2시까지 휴가
6. 김지훈: 오전 9시부터 오후 2시까지 휴가

총 6명의 직원이 오늘 휴가를 사용하고 있습니다. 정지훈님과 이선영님은 종일 휴가이며, 나머지 분들은 부분 휴가를 사용하고 있습니다.


In [31]:
# gemini
execute_agent(gemini, tools, query, "gemini")

[gemini] 결과입니다.
오늘 (2025년 4월 7일) 휴가자는 정지훈, 문승만(오후 2시~5시), 김건우(오후 1시~5시), 이선영, 천승민(오전 9시~오후 2시), 김지훈(오전 9시~오후 2시)입니다.


In [32]:
# llama3.1 70B (Together.ai)
execute_agent(
    llama,
    tools,
    query,
    "llama3.1 70B",
)

[llama3.1 70B] 결과입니다.
name": "search_dayoff", "parameters": {"start_date": "get_current_time", "end_date": "get_current_time"}}


In [33]:
# gemma3 (ollama)
execute_agent(gemma3, tools, query, "gemma3")

[gemma3] 결과입니다.
오늘(2025년 4월 7일) 휴가자는 다음과 같습니다: 정지훈, 문승만(오후 2시부터 오후 5시까지), 김건우(오후 1시부터 오후 5시까지), 이선영, 천승민(오전 9시부터 오후 2시까지), 김지훈(오전 9시부터 오후 2시까지).<|im_end|>



In [34]:
# qwen2.5 7B (ollama)
execute_agent(qwen, tools, query, "qwen2.5(Ollama)")

ResponseError: registry.ollama.ai/library/qwen-2.5:latest does not support tools (status code: 400)